In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
# pd.set_option('display.max_rows', 1000)

import numpy as np
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import itertools

%matplotlib inline

from model.env_stock_trading.env_stocktrading import StockTradingEnv
from model.stablebaselines3.models import DRLAgent
from model.plot import backtest_stats, backtest_plot, get_baseline, get_daily_return
from stable_baselines3.common.logger import configure
from helpers.missing_data import get_trading_days
from config.config import NIFTY_50, US_25
from data.database.postgres import read_table_from_database, create_connection
from data.data_extraction.feature_preprocessing import data_split, preprocess, calculate_turbulence, combine_dataframes
from helpers.missing_data import find_missing_dates
from sqlalchemy import text
import exchange_calendars as tc
import pytz


from config.config import (
    RESULTS_DIR,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    tech_list
)


In [2]:
missing_dates = find_missing_dates(start_date='2023-04-16', end_date='2023-04-30',
                                   symbols=US_25, table_name='weather_preprocessed', country='US')

In [4]:
tables_list = ['weather_preprocessed', 'daily_minute_prices',
               'technical_indicators', 'ratios', 'tweets_sentiment', 'news_sentiment']

In [ ]:
# dataframes_list = []
# for i in tables_list:

In [394]:
def get_trading_days(start: str, end: str, exchange) -> list[str]:
    nyse = tc.get_calendar(exchange)
    df = nyse.sessions_in_range(start, end)
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])

    return trading_days


trading_days = get_trading_days('2023-04-16', '2023-04-30', 'BSE')
times = []
for day in trading_days:
    IN = "Asia/Kolkata"
    current_time = pd.Timestamp(day + " 09:15:00")  # .tz_localize(IN)
    for i in range(390):  # 390 minutes in trading day
        times.append(current_time)
        current_time += pd.Timedelta(minutes=1)

In [395]:
final_df.datetime.min()

Timestamp('2023-04-17 09:15:00')

In [396]:
final_df.datetime.max()

Timestamp('2023-04-28 09:59:00')

In [397]:
final_df.drop_duplicates(subset=['datetime', 'asset_id'], inplace=True)

In [398]:
# # clean data

# tic_list = np.unique(df.asset_id.values)
# print(tic_list)
# KO = "Asia/Kolkata"

# trading_days = get_trading_days(start='2023-04-16', end='2023-04-30', exchange='BSE')
# print(trading_days)
# # produce full timestamp index
# times = []
# for day in trading_days:
#     #                NY = "America/New_York"
#     current_time = pd.Timestamp(day + " 09:15:00").tz_localize(KO)
#     for i in range(390):  # 390 minutes in trading day
#         times.append(current_time)
#         current_time += pd.Timedelta(minutes=1)
# print(times)

# # create a new dataframe with full timestamp series
# new_df = pd.DataFrame()
# for tic in tic_list:
#     tmp_df = pd.DataFrame(
#         columns=["open", "high", "low", "close", "volume"], index=times
#     )
#     tic_df = df[
#         df.asset_id == tic
#     ]  # extract just the rows from downloaded data relating to this tic
#     for i in range(tic_df.shape[0]):  # fill empty DataFrame using original data
#         print('tic_df',tic_df.iloc[i][["open", "high", "low", "close", "volume"]])
#         tmp_df.loc[tic_df.iloc[i]["datetime"].tz_localize(KO)] = tic_df.iloc[
#             i
#         ][["open", "high", "low", "close", "volume"]]
#     # print("(9) tmp_df\n", tmp_df.to_string()) # print ALL dataframe to check for missing rows from download

# #     # if close on start date is NaN, fill data with first valid close
# #     # and set volume to 0.
# #     if str(tmp_df.iloc[0]["close"]) == "nan":
# #         print("NaN data on start date, fill using first valid data.")
# #         for i in range(tmp_df.shape[0]):
# #             if str(tmp_df.iloc[i]["close"]) != "nan":
# #                 first_valid_close = tmp_df.iloc[i]["close"]
# #                 tmp_df.iloc[0] = [
# #                     first_valid_close,
# #                     first_valid_close,
# #                     first_valid_close,
# #                     first_valid_close,
# #                     0.0,
# #                 ]
# #                 break

# #     # if the close price of the first row is still NaN (All the prices are NaN in this case)
# #     if str(tmp_df.iloc[0]["close"]) == "nan":
# #         print(
# #             "Missing data for ticker: ",
# #             tic,
# #             " . The prices are all NaN. Fill with 0.",
# #         )
# #         tmp_df.iloc[0] = [
# #             0.0,
# #             0.0,
# #             0.0,
# #             0.0,
# #             0.0,
# #         ]

# #     # fill NaN data with previous close and set volume to 0.
# #     for i in range(tmp_df.shape[0]):
# #         if str(tmp_df.iloc[i]["close"]) == "nan":
# #             previous_close = tmp_df.iloc[i - 1]["close"]
# #             if str(previous_close) == "nan":
# #                 raise ValueError
# #             tmp_df.iloc[i] = [
# #                 previous_close,
# #                 previous_close,
# #                 previous_close,
# #                 previous_close,
# #                 0.0,
# #             ]
# #             # print(tmp_df.iloc[i], " Filled NaN data with previous close and set volume to 0. ticker: ", tic)

# #     # merge single ticker data to new DataFrame
# #     tmp_df = tmp_df.astype(float)
# #     tmp_df["asset_id"] = tic
# #     new_df = pd.concat([new_df, tmp_df])

# # #            print(("Data clean for ") + tic + (" is finished."))

# # # reset index and rename columns
# # new_df = new_df.reset_index()
# # new_df = new_df.rename(columns={"index": "datetime"})
# # new_df

In [399]:
# symbols = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOGL', 'NKE', 'GOOG',
#          'TSLA', 'UNH', 'META', 'XOM', 'JNJ', 'JPM', 'V', 'PG', 'HD', 'MA', 'ABBV',
#          'CVX', 'AVGO', 'MRK', 'LLY', 'PEP', 'KO', 'PFE']
symbols = ['ADANIPORTS', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL',
           'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'GRASIM', 'HCLTECH', 'HDFCBANK',
           'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'HDFC', 'ICICIBANK', 'ITC', 'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK',
           'LT', 'M&M', 'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC', 'POWERGRID', 'RELIANCE', 'SHREECEM',
           'SBIN', 'SUNPHARMA', 'TCS', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'UPL', 'ULTRACEMCO', 'WIPRO']

In [400]:
import itertools

finalDF = pd.DataFrame()
for symbol in symbols:
    # df = pd.DataFrame()
    date_symbol_combination = list(itertools.product(times, [symbol]))
    date_asset_df = pd.DataFrame(date_symbol_combination, columns=[
                                 'datetime', 'asset_id'])
    print(date_asset_df)
    print('-----------------------')
    asset_df = final_df.loc[final_df['asset_id'] == symbol]
    print(asset_df)
    print('---------------------------')
    df = pd.merge(date_asset_df, asset_df, on=[
                  'datetime', 'asset_id'], how='left')
    print(df)
    print('----------------------------')
    df = df.fillna(method='bfill').fillna(method='ffill')
    print(df)
    print('-------------------------------------------')
    finalDF = pd.concat([finalDF, df], axis=0, ignore_index=True)

                datetime    asset_id
0    2023-04-17 09:15:00  ADANIPORTS
1    2023-04-17 09:16:00  ADANIPORTS
2    2023-04-17 09:17:00  ADANIPORTS
3    2023-04-17 09:18:00  ADANIPORTS
4    2023-04-17 09:19:00  ADANIPORTS
...                  ...         ...
3895 2023-04-28 15:40:00  ADANIPORTS
3896 2023-04-28 15:41:00  ADANIPORTS
3897 2023-04-28 15:42:00  ADANIPORTS
3898 2023-04-28 15:43:00  ADANIPORTS
3899 2023-04-28 15:44:00  ADANIPORTS

[3900 rows x 2 columns]
-----------------------
        asset_id country       date  maxtempc  mintempc  pressure  visibility  \
0     ADANIPORTS   INDIA 2023-04-17      36.2      22.6    1004.8         9.8   
1     ADANIPORTS   INDIA 2023-04-17      36.2      22.6    1004.8         9.8   
2     ADANIPORTS   INDIA 2023-04-17      36.2      22.6    1004.8         9.8   
3     ADANIPORTS   INDIA 2023-04-17      36.2      22.6    1004.8         9.8   
4     ADANIPORTS   INDIA 2023-04-17      36.2      22.6    1004.8         9.8   
...          ...     

In [401]:
finalDF.datetime.max()

Timestamp('2023-04-28 15:44:00')

In [402]:
finalDF.drop(columns=['country'], inplace=True)
finalDF.rename(columns={'asset_id': 'tic', 'date': 'Date',
               'datetime': 'date'}, inplace=True)
print(finalDF.columns)
finalDF.drop_duplicates(subset=['tic', 'date'], inplace=True)
finalDF.sort_values(by=['date', 'tic'], ignore_index=True, inplace=True)
train = data_split(finalDF, '2023-04-17 09:15:00', '2023-04-28 15:44:00')
# trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
# print(len(trade))

Index(['date', 'tic', 'Date', 'maxtempc', 'mintempc', 'pressure', 'visibility',
       'humidity', 'windspeedkmph', 'precipmm',
       ...
       'vwma_50', 'sma_50', 'eps', 'opm', 'roe', 'pbt', 'pe', 'netprofit',
       'tweets_sentiment', 'news_sentiment'],
      dtype='object', length=120)
187200


In [403]:
train.loc[(train['tic'] == 'ADANIPORTS') & (train['Date'] == '2023-04-18')]

,date,tic,Date,maxtempc,mintempc,pressure,visibility,humidity,windspeedkmph,precipmm,...,vwma_50,sma_50,eps,opm,roe,pbt,pe,netprofit,tweets_sentiment,news_sentiment
375,2023-04-17 15:30:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
376,2023-04-17 15:31:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
377,2023-04-17 15:32:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
378,2023-04-17 15:33:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
379,2023-04-17 15:34:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,2023-04-18 15:25:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
761,2023-04-18 15:26:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
762,2023-04-18 15:27:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513
763,2023-04-18 15:28:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.4,0.10212,0.607513


In [404]:
for symbol in symbols:
    for day in trading_days:
        print((train.loc[(train['tic'] == symbol)
              & (train['Date'] == day)]).shape)
        print(symbol, day)
        print('-------------------')

(375, 120)
ADANIPORTS 2023-04-17
-------------------
(390, 120)
ADANIPORTS 2023-04-18
-------------------
(390, 120)
ADANIPORTS 2023-04-19
-------------------
(390, 120)
ADANIPORTS 2023-04-20
-------------------
(390, 120)
ADANIPORTS 2023-04-21
-------------------
(390, 120)
ADANIPORTS 2023-04-24
-------------------
(197, 120)
ADANIPORTS 2023-04-25
-------------------
(253, 120)
ADANIPORTS 2023-04-26
-------------------
(390, 120)
ADANIPORTS 2023-04-27
-------------------
(735, 120)
ADANIPORTS 2023-04-28
-------------------
(375, 120)
APOLLOHOSP 2023-04-17
-------------------
(390, 120)
APOLLOHOSP 2023-04-18
-------------------
(390, 120)
APOLLOHOSP 2023-04-19
-------------------
(390, 120)
APOLLOHOSP 2023-04-20
-------------------
(390, 120)
APOLLOHOSP 2023-04-21
-------------------
(390, 120)
APOLLOHOSP 2023-04-24
-------------------
(197, 120)
APOLLOHOSP 2023-04-25
-------------------
(253, 120)
APOLLOHOSP 2023-04-26
-------------------
(390, 120)
APOLLOHOSP 2023-04-27
-------------

In [406]:
train.iloc[train.index == 589]

,date,tic,Date,maxtempc,mintempc,pressure,visibility,humidity,windspeedkmph,precipmm,...,vwma_50,sma_50,eps,opm,roe,pbt,pe,netprofit,tweets_sentiment,news_sentiment
589,2023-04-18 12:34:00,ADANIPORTS,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,580.234432,617.135002,0.05,59.92,-4.51,23.27,-112.76,11.40,0.102120,0.607513
589,2023-04-18 12:34:00,APOLLOHOSP,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,4387.497929,4359.533994,17.08,28.81,15.48,323.12,62.39,245.59,0.198771,0.678725
589,2023-04-18 12:34:00,ASIANPAINT,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,2797.550920,2800.852007,11.32,22.03,27.58,1442.98,72.39,1085.88,0.092600,0.244586
589,2023-04-18 12:34:00,AXISBANK,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,853.652787,853.900996,19.04,43.05,15.60,9277.49,13.69,5853.07,0.002400,0.374489
589,2023-04-18 12:34:00,BAJAJ-AUTO,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,3899.346780,3880.461997,52.70,21.96,24.76,1963.50,21.23,1491.42,0.001911,0.654043
589,2023-04-18 12:34:00,BAJAJFINSV,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,1320.796316,1328.820010,0.00,4.69,14.27,0.71,288.51,0.49,0.000000,0.072325
589,2023-04-18 12:34:00,BAJFINANCE,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,5955.111431,5990.793008,17.74,68.63,21.21,1584.70,37.10,1022.73,0.133967,0.470371
589,2023-04-18 12:34:00,BHARTIARTL,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,763.770452,765.652003,-5.69,33.83,-5.17,-2880.50,-104.38,-3222.30,0.000000,-0.138750
589,2023-04-18 12:34:00,BPCL,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,337.084460,332.839000,9.20,3.51,-5.72,2125.84,-29.07,1959.58,0.057350,0.051913
589,2023-04-18 12:34:00,BRITANNIA,2023-04-18,36.0,22.4,1007.0,9.2,46.8,20.4,0.26,...,4409.655522,4393.939023,31.84,26.58,107.48,980.58,52.35,766.93,0.155431,0.386180


In [407]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(tech_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 48, State Space: 2785


In [408]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 50,
    "initial_amount": 100000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": tech_list,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [409]:
env_train, obs_train = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [410]:
agent = DRLAgent(env=env_train)
if_using_ppo = True

In [411]:
agent = DRLAgent(env=env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
    "target_kl": 0.05
}
model_ppo = agent.get_model("ppo", model_kwargs=PPO_PARAMS)

if if_using_ppo:
    # set up logger
    tmp_path = RESULTS_DIR + '/ppo'
    new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128, 'target_kl': 0.05}
Using cpu device
Logging to result/ppo


In [384]:
trained_ppo = agent.train_model(model=model_ppo,
                                tb_log_name='ppo',
                                total_timesteps=50000) if if_using_ppo else None

-------------------------------------
| time/              |              |
|    fps             | 39           |
|    iterations      | 1            |
|    time_elapsed    | 51           |
|    total_timesteps | 2048         |
| train/             |              |
|    reward          | -0.001803027 |
-------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 38            |
|    iterations           | 2             |
|    time_elapsed         | 106           |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.019480957   |
|    clip_fraction        | 0.229         |
|    clip_range           | 0.2           |
|    entropy_loss         | -68.2         |
|    explained_variance   | -3.56         |
|    learning_rate        | 0.00025       |
|    loss                 | -0.711        |
|    n_updates            | 10            

In [12]:
data_risk_indicator = processed_full[(processed_full.date <= TRAIN_END_DATE) & (
    processed_full.date >= TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [13]:
max(insample_risk_indicator.turbulence)

323.688337612044

In [14]:
insample_risk_indicator.turbulence.describe()

count    1723.000000
mean        9.421269
std        14.372835
min         0.000000
25%         3.155248
50%         6.688186
75%        11.284441
max       323.688338
Name: turbulence, dtype: float64

In [15]:
insample_risk_indicator.turbulence.quantile(0.995)

91.57476802750648

In [16]:
e_trade_gym = StockTradingEnv(
    df=trade, turbulence_threshold=100, risk_indicator_col='turbulence', **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

In [17]:
trained_model = trained_ppo
df_account_value, df_actions, df_rewards = DRLAgent.DRL_prediction(
    model=trained_model,
    environment=e_trade_gym)

hit end!


In [18]:
df_account_value.shape

(495, 2)

In [19]:
df_actions.shape

(494, 10)

In [20]:
df_account_value.tail(50)

,date,account_value
445,2022-10-19,12882.062818
446,2022-10-20,13097.063184
447,2022-10-21,13103.313184
448,2022-10-24,13211.863050
449,2022-10-25,13238.913282
450,2022-10-27,13325.313184
451,2022-10-28,13260.362562
452,2022-10-31,13307.762769
453,2022-11-01,13367.746533
454,2022-11-02,13397.037980


In [21]:
df_account_value['account_value'] = df_account_value['account_value'].apply(
    lambda x: '%.2f' % x)

In [22]:
df_actions.head(500)

,ADANIPORTS,APOLLOHOSP,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,BAJFINANCE,BHARTIARTL,BPCL,BRITANNIA
date,,,,,,,,,,
2021-01-01,2,1,0,0,0,0,0,0,20,0
2021-01-04,-2,0,0,0,0,1,0,1,-1,0
2021-01-05,0,-1,0,0,0,0,0,-1,9,0
2021-01-06,0,0,0,0,0,-1,0,0,2,0
2021-01-07,0,0,0,0,0,0,0,0,0,0
2021-01-08,0,0,0,0,0,2,0,0,-6,0
2021-01-11,0,0,0,0,0,0,0,0,1,0
2021-01-12,0,0,0,0,0,-2,0,7,-7,0
2021-01-13,0,0,0,0,0,0,0,0,1,0


In [23]:
predictions = pd.merge(df_actions.reset_index(), df_account_value, on=[
                       'date'], how='left').merge(df_rewards, on=['date'], how='left')
predictions

,date,ADANIPORTS,APOLLOHOSP,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,BAJFINANCE,BHARTIARTL,BPCL,BRITANNIA,account_value,rewards
0,2021-01-01,2,1,0,0,0,0,0,0,20,0,10000.00,201.505848
1,2021-01-04,-2,0,0,0,0,1,0,1,-1,0,10201.51,32.295080
2,2021-01-05,0,-1,0,0,0,0,0,-1,9,0,10233.80,-120.763244
3,2021-01-06,0,0,0,0,0,-1,0,0,2,0,10113.04,122.816328
4,2021-01-07,0,0,0,0,0,0,0,0,0,0,10235.85,201.746521
5,2021-01-08,0,0,0,0,0,2,0,0,-6,0,10437.60,-8.458870
6,2021-01-11,0,0,0,0,0,0,0,0,1,0,10429.14,11.160189
7,2021-01-12,0,0,0,0,0,-2,0,7,-7,0,10440.30,221.361747
8,2021-01-13,0,0,0,0,0,0,0,0,1,0,10661.66,253.548179
9,2021-01-14,0,0,0,0,0,0,0,0,0,0,10915.21,-40.457458


In [24]:
predict = processed_full.iloc[-20:, :]
predict

,aroond_14,aroonosc_14,aroonu_14,bbb_5_2.0,bbl_5_2.0,bbm_5_2.0,bbp_5_2.0,bbu_5_2.0,cdl_doji_10_0.1,cdl_inside,...,tweets_sentiment,visibility,volume,vwma_10,vwma_20,vwma_50,windspeedkmph,wma_10,wma_20,wma_50
22580,85.714286,-78.571429,7.142857,8.878574,535.703804,560.590002,0.526320,585.476201,0.0,0,...,0.048578,9.4,3442489,566.653105,530.415168,583.234828,11.0,564.376369,564.022624,634.472671
22581,42.857143,14.285714,57.142857,2.236382,4405.634578,4455.455078,0.199370,4505.275578,0.0,0,...,0.102120,9.4,88243,4510.111756,4437.345100,4455.698744,11.0,4485.419114,4448.857645,4393.283776
22582,85.714286,-21.428571,64.285714,6.145853,2672.201898,2756.920020,0.324005,2841.638141,0.0,-1,...,-0.089100,9.4,204058,2771.328400,2761.634506,2868.145236,11.0,2768.520930,2776.729781,2821.031257
22583,78.571429,-78.571429,0.000000,2.865979,832.102692,844.199988,0.650860,856.297283,0.0,0,...,-0.100568,9.4,2516315,852.583535,862.349444,899.258866,11.0,848.156357,856.013804,883.276742
22584,100.000000,-35.714286,64.285714,6.176448,3692.958896,3810.639990,0.002512,3928.321084,0.0,0,...,-0.053818,9.4,358152,3840.664990,3842.351921,3749.153051,11.0,3830.011821,3847.799769,3773.409882
22585,0.000000,50.000000,50.000000,4.740593,1331.122418,1363.439990,0.384892,1395.757563,100.0,0,...,-0.237867,9.4,331051,1393.519819,1355.970690,1407.772793,11.0,1375.691810,1379.263334,1381.394195
22586,0.000000,21.428571,21.428571,4.856420,6084.108490,6235.520020,0.254246,6386.931549,0.0,0,...,0.023062,9.4,161217,6341.473426,6155.946826,6140.299423,11.0,6277.752752,6297.352402,6174.354120
22587,100.000000,-100.000000,0.000000,6.167557,739.751831,763.290002,0.115731,786.828174,0.0,0,...,-0.088080,9.4,3056050,771.340669,773.958863,781.205952,11.0,766.384550,771.395719,776.828513
22588,100.000000,-71.428571,28.571429,2.542183,315.912761,319.979999,0.268886,324.047237,100.0,0,...,0.129694,9.4,591543,324.720625,331.154097,336.410324,11.0,322.074545,326.253810,333.764629
22589,100.000000,-78.571429,21.428571,2.694160,4400.156943,4460.239941,0.159303,4520.322939,0.0,0,...,-0.008758,9.4,41922,4507.500917,4527.239117,4441.108980,11.0,4474.441752,4520.904267,4463.047212


In [25]:
predict.date.values[0]

numpy.datetime64('2023-02-27T00:00:00.000000000')

In [26]:
predict = predict.sort_values(['date', "tic"], ignore_index=True)
predict.index = predict['date'].factorize()[0]
predict

,aroond_14,aroonosc_14,aroonu_14,bbb_5_2.0,bbl_5_2.0,bbm_5_2.0,bbp_5_2.0,bbu_5_2.0,cdl_doji_10_0.1,cdl_inside,...,tweets_sentiment,visibility,volume,vwma_10,vwma_20,vwma_50,windspeedkmph,wma_10,wma_20,wma_50
0,85.714286,-78.571429,7.142857,8.878574,535.703804,560.590002,0.526320,585.476201,0.0,0,...,0.048578,9.4,3442489,566.653105,530.415168,583.234828,11.0,564.376369,564.022624,634.472671
0,42.857143,14.285714,57.142857,2.236382,4405.634578,4455.455078,0.199370,4505.275578,0.0,0,...,0.102120,9.4,88243,4510.111756,4437.345100,4455.698744,11.0,4485.419114,4448.857645,4393.283776
0,85.714286,-21.428571,64.285714,6.145853,2672.201898,2756.920020,0.324005,2841.638141,0.0,-1,...,-0.089100,9.4,204058,2771.328400,2761.634506,2868.145236,11.0,2768.520930,2776.729781,2821.031257
0,78.571429,-78.571429,0.000000,2.865979,832.102692,844.199988,0.650860,856.297283,0.0,0,...,-0.100568,9.4,2516315,852.583535,862.349444,899.258866,11.0,848.156357,856.013804,883.276742
0,100.000000,-35.714286,64.285714,6.176448,3692.958896,3810.639990,0.002512,3928.321084,0.0,0,...,-0.053818,9.4,358152,3840.664990,3842.351921,3749.153051,11.0,3830.011821,3847.799769,3773.409882
0,0.000000,50.000000,50.000000,4.740593,1331.122418,1363.439990,0.384892,1395.757563,100.0,0,...,-0.237867,9.4,331051,1393.519819,1355.970690,1407.772793,11.0,1375.691810,1379.263334,1381.394195
0,0.000000,21.428571,21.428571,4.856420,6084.108490,6235.520020,0.254246,6386.931549,0.0,0,...,0.023062,9.4,161217,6341.473426,6155.946826,6140.299423,11.0,6277.752752,6297.352402,6174.354120
0,100.000000,-100.000000,0.000000,6.167557,739.751831,763.290002,0.115731,786.828174,0.0,0,...,-0.088080,9.4,3056050,771.340669,773.958863,781.205952,11.0,766.384550,771.395719,776.828513
0,100.000000,-71.428571,28.571429,2.542183,315.912761,319.979999,0.268886,324.047237,100.0,0,...,0.129694,9.4,591543,324.720625,331.154097,336.410324,11.0,322.074545,326.253810,333.764629
0,100.000000,-78.571429,21.428571,2.694160,4400.156943,4460.239941,0.159303,4520.322939,0.0,0,...,-0.008758,9.4,41922,4507.500917,4527.239117,4441.108980,11.0,4474.441752,4520.904267,4463.047212


In [27]:
predict_env = StockTradingEnv(
    df=predict, turbulence_threshold=100, risk_indicator_col='turbulence', **env_kwargs)

In [35]:
predict.replace('--', 0, inplace=True)

In [37]:
cwd = 'trained_model/PPO.zip'
assets, actions, rewards = DRLAgent.DRL_prediction_load_from_file(
    'ppo', predict_env, cwd, deterministic=True)

hit end!


In [38]:
assets

,date,account_value
0,2023-02-27,10000.000000
1,2023-02-28,10016.395109


In [39]:
actions

,ADANIPORTS,APOLLOHOSP,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,BAJFINANCE,BHARTIARTL,BPCL,BRITANNIA
date,,,,,,,,,,
2023-02-27,0,2,0,0,0,0,0,0,3,0


In [40]:
rewards

,date,rewards
0,2023-02-27,16.395109


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
df_account_value

,date,account_value
0,2021-01-01,1000000.00
1,2021-01-04,999967.80
2,2021-01-05,999798.62
3,2021-01-06,999538.50
4,2021-01-07,999928.83
5,2021-01-08,1000164.19
6,2021-01-11,999315.22
7,2021-01-12,1000072.15
8,2021-01-13,998479.00
9,2021-01-14,998239.67


In [ ]:
# baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
    ticker="BPCL.NS",
    start=df_account_value.loc[0, 'date'],
    end=df_account_value.loc[len(df_account_value)-1, 'date'])

stats = backtest_stats(baseline_df, value_col_name='close')

==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (245, 8)
Annual return          0.145942
Cumulative returns     0.141614
Annual volatility      0.261816
Sharpe ratio           0.653568
Calmar ratio           0.686717
Stability              0.204899
Max drawdown          -0.212521
Omega ratio            1.115651
Sortino ratio          0.916846
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.989570
Daily value at risk   -0.032307
dtype: float64


/home/bhumika/Desktop/FinRL-master/finrl/meta/preprocessor/yahoodownloader.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/home/bhumika/.local/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [ ]:
df_account_value.loc[0, 'date']

datetime.date(2021, 1, 1)

In [ ]:
df_account_value.loc[len(df_account_value)-1, 'date']

datetime.date(2021, 12, 29)

In [ ]:
%matplotlib inline
backtest_plot(df_account_value, 
             baseline_ticker = 'BPCL.NS', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

==============Compare to DJIA===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (245, 8)


/home/bhumika/Desktop/FinRL-master/finrl/meta/preprocessor/yahoodownloader.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/home/bhumika/.local/lib/python3.8/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/home/bhumika/.local/lib/python3.8/site-packages/pyfolio/plotting.py:648: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


Start date,2021-01-01
End date,2021-12-29
Total months,11
,Backtest
Annual return,27.9%
Cumulative returns,26.3%
Annual volatility,21.5%
Sharpe ratio,1.26
Calmar ratio,1.89
Stability,0.80
Max drawdown,-14.8%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'